# Belegungsquote der Frauenhäuser in Deutschland

## Cleaning up original data

In [1]:
import pandas as pd
import numpy as np

In [2]:
fpath = "data/scraped/230103_datenstand.json"

In [3]:
# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

In [4]:
# get keys
keys = pd.read_csv("data/helpers/free_places_key.csv")

In [5]:
# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])

In [6]:
# reformat timeseries
df.timestamp = pd.to_datetime(df.timestamp, dayfirst=True)
df["date"] = df.timestamp.dt.date

In [7]:
# add text keys
df.loc[df.freePlaces=="", "freePlaces"] = np.nan
df.freePlaces = df.freePlaces.astype(float)
df = pd.merge(df, keys, on="freePlaces")

# fill nas
df.loc[df.description.isnull(), "description"] = "k.A."

# add simplified NAs
desc_short = {
    "Aufnahme möglich für Frauen mit 4 oder mehr Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen mit 3 Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen mit 2 Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen ohne Kinder, Aufnahme möglich für Frauen mit 1 Kind":"Aufnahme möglich für Frauen mit bis zu einem Kind",
    "Aufnahme möglich für Frauen ohne Kinder":"Platz für Frauen ohne Kinder",
    "Keine Aufnahme möglich":"Keine Aufnahme möglich",
    "Aufnahme möglich ohne detaillierte Angabe":"Aufnahme möglich ohne detaillierte Angabe",
    "k.A.":"keine Angabe"
}

df["desc_short"] = df.description.map(desc_short)

In [8]:
# add bundesland
geocoded = pd.read_csv("data/helpers/shelters_geocoded.csv").drop(["latitude","longitude"], axis=1)
df = pd.merge(df, geocoded, on=["shelter_name","shelter_id"])

In [9]:
df.head()

,timestamp,freePlaces,shelter_id,shelter_name,latitude,longitude,date,description,desc_short,gen,bez,nuts,nuts2,bundesland
0,2021-12-17 18:00:00,4.0,2270,Frauen- und Kinderschutzhaus Aachen,50.788047,6.097588,2021-12-17,Aufnahme möglich für Frauen mit 3 Kindern,Aufnahme möglich für Frauen mit mehreren Kindern,Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen
1,2021-12-22 18:01:00,4.0,2270,Frauen- und Kinderschutzhaus Aachen,50.788047,6.097588,2021-12-22,Aufnahme möglich für Frauen mit 3 Kindern,Aufnahme möglich für Frauen mit mehreren Kindern,Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen
2,2021-12-23 08:01:00,4.0,2270,Frauen- und Kinderschutzhaus Aachen,50.788047,6.097588,2021-12-23,Aufnahme möglich für Frauen mit 3 Kindern,Aufnahme möglich für Frauen mit mehreren Kindern,Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen
3,2022-02-21 12:01:00,4.0,2270,Frauen- und Kinderschutzhaus Aachen,50.788047,6.097588,2022-02-21,Aufnahme möglich für Frauen mit 3 Kindern,Aufnahme möglich für Frauen mit mehreren Kindern,Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen
4,2022-05-05 18:01:00,4.0,2270,Frauen- und Kinderschutzhaus Aachen,50.788047,6.097588,2022-05-05,Aufnahme möglich für Frauen mit 3 Kindern,Aufnahme möglich für Frauen mit mehreren Kindern,Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen


## Monthly shelter-level summary

In [10]:
# first get one value per day based on most common
df_shelter_date = df.groupby(["shelter_name","shelter_id","bundesland","bez","gen","date","desc_short","latitude","longitude"]).agg(
    n = ("desc_short", "count")
).reset_index().sort_values(by=["shelter_id","date"], ascending=False) \
.groupby(["shelter_name","shelter_id","bundesland","bez","gen","latitude","longitude","date"]).agg(
    status = ("desc_short", "first")
).reset_index()

In [11]:
# get month-year
df_shelter_date["date"] = pd.to_datetime(df_shelter_date["date"])
df_shelter_date["monthyear"] = df_shelter_date.date.dt.to_period("M")
df_shelter_date.head()

,shelter_name,shelter_id,bundesland,bez,gen,latitude,longitude,date,status,monthyear
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.99664,2021-12-17,keine Angabe,2021-12
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.99664,2021-12-20,keine Angabe,2021-12
2,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.99664,2021-12-21,keine Angabe,2021-12
3,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.99664,2021-12-22,keine Angabe,2021-12
4,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.99664,2021-12-23,keine Angabe,2021-12


In [12]:
df_shelter_monthyear = df_shelter_date.groupby(["shelter_name","shelter_id","bundesland","bez","gen","monthyear","status"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter_monthyear.head()

,shelter_name,shelter_id,bundesland,bez,gen,monthyear,status,n
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2021-12,keine Angabe,13
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-01,keine Angabe,31
2,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-02,keine Angabe,28
3,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-03,keine Angabe,31
4,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-04,keine Angabe,30


In [13]:
# pivot to wide and fill values
df_shelter_monthly = pd.pivot(
    df_shelter_monthyear,
    index=["shelter_name","shelter_id","bundesland","bez","gen","monthyear"],
    columns="status",
    values="n"
).reset_index().replace(np.nan, 0)
df_shelter_monthly.head()

status,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2021-12,0.0,0.0,0.0,0.0,0.0,13.0
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-01,0.0,0.0,0.0,0.0,0.0,31.0
2,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-02,0.0,0.0,0.0,0.0,0.0,28.0
3,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-03,0.0,0.0,0.0,0.0,0.0,31.0
4,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-04,0.0,0.0,0.0,0.0,0.0,30.0


In [14]:
# calculate total
df_shelter_monthly["n"] = df_shelter_monthly.iloc[:,6:].sum(axis=1)

In [15]:
# get columns as pct
df_shelter_monthly.iloc[:,6:-1] = df_shelter_monthly.iloc[:,6:-1].apply(lambda x : x / df_shelter_monthly.n, axis=0)

In [16]:
df_shelter_monthly = df_shelter_monthly.drop("n", axis=1)
df_shelter_monthly.to_csv("./data/cleaned/belegungsquote_nach_year_month_shelter.csv", index=False)

## Overall shelter-level summary

In [17]:
df_shelter = df_shelter_date.groupby(["shelter_name","shelter_id","bundesland","bez","gen","status","latitude","longitude"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter.head()

,shelter_name,shelter_id,bundesland,bez,gen,status,latitude,longitude,n
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,Aufnahme möglich ohne detaillierte Angabe,53.552828,9.996640,7
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,keine Angabe,53.552828,9.996640,374
2,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Aufnahme möglich für Frauen mit bis zu einem Kind,51.325188,12.373901,51
3,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Keine Aufnahme möglich,51.325188,12.373901,93
4,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Platz für Frauen ohne Kinder,51.325188,12.373901,237


In [18]:
# pivot to wide and fill values
df_shelter = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id","bundesland","bez","gen","latitude","longitude"],
    columns="status",
    values="n"
).reset_index().replace(np.nan, 0)

# calculate total
df_shelter["n"] = df_shelter.iloc[:,7:].sum(axis=1)

# get columns as pct
df_shelter.iloc[:,7:-1] = df_shelter.iloc[:,7:-1].apply(lambda x : x / df_shelter.n, axis=0)

df_shelter = df_shelter.drop("n", axis=1)
df_shelter.to_csv("./data/cleaned/belegungsquote_nach_shelter.csv", index=False)

## Get all shelters with 50% or more keine Angabe

In [19]:
no_data = df_shelter.loc[df_shelter["keine Angabe"] >= 0.50, "shelter_id"].values

## Gesamtübersicht nach Monat

In [20]:
monthly = df_shelter_date.loc[~df_shelter_date.shelter_id.isin(no_data),].groupby(["monthyear","status"]).agg(
    n = ("status", "count")
).reset_index()

monthly = pd.pivot(
    monthly,
    index=["monthyear"],
    columns="status",
    values="n"
).reset_index().replace(np.nan, 0)

monthly["n"] = monthly.iloc[:,1:].sum(axis=1)
monthly.iloc[:,1:-1] = monthly.iloc[:,1:-1].apply(lambda x : x / monthly.n, axis=0)

monthly = monthly.drop("n", axis=1)

monthly.head()

status,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,2021-12,0.017282,0.124703,0.100305,0.689597,0.010505,0.057608
1,2022-01,0.020397,0.136756,0.117728,0.650103,0.020808,0.054209
2,2022-02,0.024321,0.114547,0.100585,0.693590,0.023420,0.043537
3,2022-03,0.021259,0.094922,0.101422,0.708057,0.026540,0.047800
4,2022-04,0.022365,0.088063,0.086385,0.744618,0.025720,0.032849


In [21]:
monthly.to_csv("./data/cleaned/monthly_overview.csv", index=False)

## Gesamtübersicht nach Bundesland

In [22]:
# filter out where no data and mean of each status % by bundesland
nach_bundesland = df_shelter.loc[~df_shelter.shelter_id.isin(no_data),] \
    .groupby(["bundesland"]) \
    .mean() \
    .reset_index()

nach_bundesland = nach_bundesland.drop(["shelter_id","latitude","longitude"], axis=1)

nach_bundesland

status,bundesland,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,Baden-Württemberg,0.016948,0.099388,0.051865,0.724677,0.004944,0.102178
1,Bayern,0.012039,0.136222,0.069179,0.703796,0.011717,0.067047
2,Berlin,0.005249,0.032808,0.000000,0.814961,0.062992,0.083990
3,Brandenburg,0.102362,0.182415,0.021654,0.656168,0.000656,0.036745
4,Bremen,0.000000,0.063063,0.000000,0.936937,0.000000,0.000000
5,Hessen,0.013901,0.035579,0.023914,0.923496,0.003111,0.000000
6,Mecklenburg-Vorpommern,0.010810,0.070077,0.107028,0.642647,0.036454,0.132983
7,Niedersachsen,0.015715,0.102279,0.142724,0.665847,0.032945,0.040490
8,Nordrhein-Westfalen,0.014580,0.039918,0.067549,0.870588,0.007366,0.000000
9,Rheinland-Pfalz,0.023959,0.018256,0.027184,0.920701,0.009137,0.000763


In [23]:
nach_bundesland.to_csv("./data/cleaned/bundesland_overview.csv", index=False)

## Weihnachts Beispiel - 12.25.2022 um 8 Uhr

In [24]:
weihnacht = df.loc[df.timestamp==pd.to_datetime("2022-25-12 08:01:00", dayfirst=True),] # filter by day
weihnacht = weihnacht.loc[~weihnacht.shelter_id.isin(no_data),] # remove where no data overall
weihnacht = weihnacht.loc[weihnacht.description != "k.A.",] # remove where no data for that day
weihnacht = weihnacht[["shelter_name","shelter_id","latitude","longitude","timestamp","description","desc_short","gen","bez","bundesland"]]
weihnacht.to_csv("./data/cleaned/belegungsstatus_25-12-2022_8-01.csv", index=False)